In [1]:
import pandas as pd
from bertopic import BERTopic



/Users/camille/repo/Hetic/projet_gouv/scraping/clean_data_env/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/camille/repo/Hetic/projet_gouv/scraping/clean_data_env/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/camille/r

In [2]:
data = pd.read_csv("/Users/camille/repo/Hetic/projet_gouv/scraping/data_nlp/google_reviews_RGPD.csv")
data.dropna(subset=['review_text'], inplace=True)

In [3]:
data

,date,rate,review_text,object_address,review_num
0,2 months ago,1,Pas de formation possible\nPas de financement ...,"5 Rue de Quimper, 68100 Mulhouse",32
1,a month ago,1,"Aucun respect, j’explique pourtant bien mon so...","5 Rue de Quimper, 68100 Mulhouse",32
2,4 months ago,1,"Les fonctionnaires incompétents, arrogants et ...","5 Rue de Quimper, 68100 Mulhouse",32
3,9 months ago,1,"Aucun respect, j’explique pourtant bien mon so...","5 Rue de Quimper, 68100 Mulhouse",32
4,9 months ago,1,Si je pouvais ne pas mettre d'étoiles je n'en ...,"5 Rue de Quimper, 68100 Mulhouse",32
...,...,...,...,...,...
16623,8 years ago,3,j'ai eu soixante ans le 15 mars ; suis je obli...,"45 Av. Billaud Varenne, 17000 La Rochelle",34
16624,6 years ago,3,Un pôle emploi comme un autre,"45 Av. Billaud Varenne, 17000 La Rochelle",34
16625,a year ago,1,Incompétents.,"45 Av. Billaud Varenne, 17000 La Rochelle",34
16626,3 years ago,1,Sourires en option,"45 Av. Billaud Varenne, 17000 La Rochelle",34


In [5]:
data['pos'] = data['rate'].apply(lambda x:  x >= 2.5)

In [17]:
data['pos'] = data['rate'].apply(lambda x:  x >= 2.5)
df_pos = data[data['pos'] == True].reset_index(drop=True)
df_neg = data[data['pos'] == False].reset_index(drop=True)

In [18]:
data = data.reset_index(drop=True)

# Instanciez le modèle BERTopic
topic_model = BERTopic()
BERTopic(language="multilingual") 
# Transformez les données du DataFrame
topics, probs = topic_model.fit_transform(df_neg['review_text'])

In [19]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,2352,-1_de_et_je_pas
1,0,287,0_emploi_ple_pole_les
2,1,261,1_incomptent_incomptents_incomptence_personnel
3,2,257,2_que_je_de_et
4,3,175,3_tlphone_au_impossible_pas
...,...,...,...
73,72,12,72_agents_agent_guichet_mchancet
74,73,11,73_site_crme_sites_anpe
75,74,10,74_personne_perturb_extrmement_lentretien
76,75,10,75_rappeler_2h_rappel_maccorder


In [20]:
topic_model.get_topic(0)


[('emploi', 0.036756581508663864),
 ('ple', 0.03262550855661239),
 ('pole', 0.026128081246442104),
 ('les', 0.012072759949087783),
 ('cest', 0.011606627324831661),
 ('de', 0.011474805369512217),
 ('le', 0.011240404243618123),
 ('pire', 0.011131020459224693),
 ('pas', 0.010695574207665957),
 ('ne', 0.009398769816440883)]

In [4]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
topic_model = BERTopic(embedding_model=sentence_model)

In [55]:
topics, probs = topic_model.fit_transform(data['review_text'])

In [22]:
hierarchical_topics = topic_model.hierarchical_topics(df_neg['review_text'])


100%|██████████| 76/76 [00:00<00:00, 355.78it/s]


In [23]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


In [31]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load("fr_core_news_sm", exclude=['tagger', 'parser', 'ner', 
                                             'attribute_ruler', 'lemmatizer'])

In [32]:
topic_model = BERTopic(embedding_model=nlp)
topics, probs = topic_model.fit_transform(df_neg['review_text'])


In [66]:
topic_model.get_topic_info()
topic_model.get_topic(3)

[('ferm', 0.06701690878348683),
 ('le', 0.05354249490379142),
 ('ferme', 0.04854751460007291),
 ('et', 0.04713419756182352),
 ('fermeture', 0.04612118187822369),
 ('horaires', 0.04438091646155777),
 ('ouvert', 0.04234104234281196),
 ('12h30', 0.038921016690910526),
 ('sur', 0.03629156480155074),
 ('fermer', 0.03479696822339823)]

In [41]:
from bertopic.vectorizers import ClassTfidfTransformer

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
topic_model = BERTopic(ctfidf_model=ctfidf_model)
topics, probs = topic_model.fit_transform(df_neg['review_text'])

In [75]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,2567,-1_et_pas_je_que
1,0,2757,0_et_pas_je_que
2,1,174,1_numro_nul_3949_le
3,2,96,2_parking_pas_pour_et
4,3,86,3_ferm_le_ferme_et
5,4,73,4_documents_scanner_document_les
6,5,52,5_radiation_pour_les_vous
7,6,42,6_dossier_dossiers_pour_les
8,7,24,7_catastrophique_des_la_wesh
9,8,13,8_good_notgood_hhhhhhh_hh


In [44]:
import spacy
sp = spacy.load("fr_core_news_sm")
spacy_stopwords = sp.Defaults.stop_words


In [60]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words=spacy_stopwords)
topic_model = BERTopic(vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(df_neg['review_text'])



In [61]:
topic_labels = topic_model.generate_topic_labels(nr_words=2, separator=", ")


In [68]:
topic_model.reduce_topics(df_neg['review_text'], nr_topics=20)


In [69]:
topics = topic_model.topics_
probabilities = topic_model.probabilities_

In [50]:
text = "Salut j'ai un chat"
text_without_stopword = [word for word in text if word not in spacy_stopwords]

In [51]:
text_without_stopword

['S', 'l', 'u', 't', ' ', 'j', "'", ' ', 'u', 'n', ' ', 'c', 'h', 't']

In [70]:
topic_model.visualize_topics()


In [32]:
topic_model.get_document_info(data['review_text'])

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,Pas de formation possible\nPas de financement ...,36,36_financement_formation_financer_mon,financement - formation - financer - mon - ma ...,0.959752,False
1,"Aucun respect, j’explique pourtant bien mon so...",-1,-1_de_et_pour_je,de - et - pour - je - les - emploi - que - pas...,0.000000,False
2,"Les fonctionnaires incompétents, arrogants et ...",3,3_incompétents_incompétent_des_vous,incompétents - incompétent - des - vous - ne -...,0.613244,False
3,"Aucun respect, j’explique pourtant bien mon so...",-1,-1_de_et_pour_je,de - et - pour - je - les - emploi - que - pas...,0.000000,False
4,Si je pouvais ne pas mettre d'étoiles je n'en ...,-1,-1_de_et_pour_je,de - et - pour - je - les - emploi - que - pas...,0.000000,False
...,...,...,...,...,...,...
9533,j'ai eu soixante ans le 15 mars ; suis je obli...,5,5_emploi_pôle_pole_est,emploi - pôle - pole - est - le - un - de - je...,1.000000,False
9534,Un pôle emploi comme un autre,5,5_emploi_pôle_pole_est,emploi - pôle - pole - est - le - un - de - je...,1.000000,False
9535,Incompétents.,41,41_incompétent_incompétents_useless_incompetent,incompétent - incompétents - useless - incompe...,0.179138,False
9536,Sourires en option,-1,-1_de_et_pour_je,de - et - pour - je - les - emploi - que - pas...,0.000000,False


In [46]:
#positive reviews superieur à 2,5
data['pos'] = data['rate'].apply(lambda x:  x >= 2.5)

In [47]:
data

,date,rate,review_text,object_address,review_num,pos
0,2 months ago,1,Pas formation possible Pas financement possibl...,"5 Rue de Quimper, 68100 Mulhouse",32,False
1,a month ago,1,"Aucun respect, j’explique pourtant bien soucis...","5 Rue de Quimper, 68100 Mulhouse",32,False
2,4 months ago,1,"Les fonctionnaires incompétents, arrogants pré...","5 Rue de Quimper, 68100 Mulhouse",32,False
3,9 months ago,1,"Aucun respect, j’explique pourtant bien soucis...","5 Rue de Quimper, 68100 Mulhouse",32,False
4,9 months ago,1,"Si pouvais mettre d'étoiles n'en mettrais pas,...","5 Rue de Quimper, 68100 Mulhouse",32,False
...,...,...,...,...,...,...
9533,8 years ago,3,j'ai soixante ans 15 mars ; obligé pointé a Po...,"45 Av. Billaud Varenne, 17000 La Rochelle",34,True
9534,6 years ago,3,Un pôle emploi comme autre,"45 Av. Billaud Varenne, 17000 La Rochelle",34,True
9535,a year ago,1,Incompétents.,"45 Av. Billaud Varenne, 17000 La Rochelle",34,False
9536,3 years ago,1,Sourires option,"45 Av. Billaud Varenne, 17000 La Rochelle",34,False


In [48]:
pos_df = data[data['pos'] == True]
neg_df = data[data['pos'] == False]

In [84]:
neg_df = pd.DataFrame(neg_df['review_text'])

In [103]:
text_neg = ''.join(neg_df['review_text'].tolist())


In [104]:
text_neg

'Pas formation possible Pas financement possible refus refus parcours Et rendez régulier inutile. Dire disant Aider chômeurs a être réactif. Absurde !Aucun respect, j’explique pourtant bien soucis fait exprès comprendre faire tourner rond. Pourtant tout document fourni.Les fonctionnaires incompétents, arrogants prétentieux, transformé pôle emploi comme bunker, traitant pauvres chômeurs comme citoyens cinquième classe, obligeant mendier droits.Aucun respect, j’explique pourtant bien soucis fait exprès comprendre. Un air hautain discriminant. Je souhaite personnes rencontrer quelconque soucis vie, moins qu’ils accumulent tellement c’est façon venger vies médiocres, ennuyeuses, déprimantes.Si pouvais mettre d\'étoiles n\'en mettrais pas, conseillère indemnisations hyper désagréable. Des j\'ai décroché téléphone a agressive air agacé. J\'ai voulu qu\'elle rappel avoir info plus échange mail. Elle a méprisante cite : "mais l\'ai écris mail, qu\'est comprenez pas?" permet juger gens faire co

In [1]:
#Bert sur les reviews negatives
topic_model = BERTopic()
BERTopic(language="multilingual") 
# Transformez les données du DataFrame
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
topic_model = BERTopic(embedding_model=sentence_model)




NameError: name 'BERTopic' is not defined

In [109]:
topics, probs = topic_model.fit_transform(text_neg)

TypeError: Make sure that the documents variable is an iterable containing strings only.

In [107]:
topics, probs = topic_model.fit_transform(text_neg)

TypeError: Make sure that the documents variable is an iterable containing strings only.

In [42]:
# Remove french stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('french')
data['review_text'] = data['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


Coun